In [1]:
%pip install transformers datasets

In [2]:
import pandas as pd

df = pd.read_csv('/content/reply_classification_dataset - reply_classification_dataset.csv')
display(df.head())
display(df.info())

,reply,label
0,Can we discuss pricing??,NEUTRAL
1,"Im excited to explore this further, plz send c...",POSITIVE
2,We not looking for new solutions.,negative
3,Could u clarify features included?,neutral
4,"lets,, schedule a meeting to dive deeper",positive


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   reply   2129 non-null   object
 1   label   2129 non-null   object
dtypes: object(2)
memory usage: 33.4+ KB


None

In [3]:
from transformers import DistilBertTokenizerFast
from datasets import Dataset

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

label_map = {label: i for i, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_map)

tokenized_data = tokenizer(list(df['reply']), padding=True, truncation=True, return_tensors='pt')

dataset_dict = {'input_ids': tokenized_data['input_ids'],
                'attention_mask': tokenized_data['attention_mask'],
                'labels': list(df['label'])}

dataset = Dataset.from_dict(dataset_dict)

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Training dataset size: 1703
Validation dataset size: 426


In [4]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_map))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
)

print(training_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_use_gather_object=False,


In [7]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prachipandeyk85 (prachipandeyk85-birla-institute-of-technology-mesra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.067100
20,1.596000
30,1.124200
40,1.064200
50,0.816400
60,0.718300
70,0.643500
80,0.722300
90,0.638700
100,0.448900


TrainOutput(global_step=639, training_loss=0.5245750334714313, metrics={'train_runtime': 693.1268, 'train_samples_per_second': 7.371, 'train_steps_per_second': 0.922, 'total_flos': 18507902248836.0, 'train_loss': 0.5245750334714313, 'epoch': 3.0})

In [8]:
trainer.save_model("./results")

In [9]:
from sklearn.metrics import accuracy_score, f1_score

predictions = trainer.predict(val_dataset)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [10]:
import numpy as np

predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.8098591549295775
F1 Score: 0.8115560732309774


In [13]:
!zip -r results.zip results
from google.colab import files
files.download("results.zip")


  adding: results/ (stored 0%)
  adding: results/config.json (deflated 55%)
  adding: results/model.safetensors (deflated 8%)
  adding: results/checkpoint-639/ (stored 0%)
  adding: results/checkpoint-639/optimizer.pt (deflated 41%)
  adding: results/checkpoint-639/config.json (deflated 55%)
  adding: results/checkpoint-639/model.safetensors (deflated 8%)
  adding: results/checkpoint-639/trainer_state.json (deflated 77%)
  adding: results/checkpoint-639/rng_state.pth (deflated 26%)
  adding: results/checkpoint-639/scheduler.pt (deflated 62%)
  adding: results/checkpoint-639/training_args.bin (deflated 53%)
  adding: results/training_args.bin (deflated 53%)
  adding: results/checkpoint-500/ (stored 0%)
  adding: results/checkpoint-500/optimizer.pt (deflated 41%)
  adding: results/checkpoint-500/config.json (deflated 55%)
  adding: results/checkpoint-500/model.safetensors (deflated 8%)
  adding: results/checkpoint-500/trainer_state.json (deflated 77%)
  adding: results/checkpoint-500/rng

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>